In [23]:
import pandas as pd
import numpy as np
import ta

In [24]:
df = pd.read_csv('sp500_ohlcv_2008_2023.csv')  
df.reset_index(drop=True, inplace=True)

for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [25]:
# Convert Close to numeric, invalid parsing will become NaN
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

# Now calculate returns
df['Return'] = df['Close'].pct_change(fill_method=None)


In [26]:
df['Target'] = np.where(df['Return'].shift(-1) > 0, 1, 0)


In [27]:
for lag in range(1, 6):
    df[f'Return_lag{lag}'] = df['Return'].shift(lag)
    df[f'Volume_lag{lag}'] = df['Volume'].shift(lag)


In [28]:
# SMA
df['SMA_10'] = ta.trend.sma_indicator(df['Close'], window=10)
# EMA
df['EMA_10'] = ta.trend.ema_indicator(df['Close'], window=10)
# RSI
df['RSI_14'] = ta.momentum.rsi(df['Close'], window=14)
# MACD
df['MACD'] = ta.trend.macd(df['Close'])
df['MACD_signal'] = ta.trend.macd_signal(df['Close'])
# Bollinger Bands
bb = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
df['BB_High'] = bb.bollinger_hband()
df['BB_Low'] = bb.bollinger_lband()
# ATR
df['ATR_14'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'], window=14)


In [29]:
df = df.dropna().reset_index(drop=True)


In [30]:
print(df.head(5))

        Date        Close         High          Low         Open      Volume  \
0  2/20/2008  1360.030029  1363.709961  1336.550049  1348.390015  3870520000   
1  2/21/2008  1342.530029  1367.939941  1339.339966  1362.209961  3696660000   
2  2/22/2008  1353.109985  1354.300049  1327.040039  1344.219971  3572660000   
3  2/25/2008  1371.800049  1374.359985  1346.030029  1352.750000  3866350000   
4  2/26/2008  1381.290039  1387.339966  1363.290039  1371.760010  4096060000   

     Return  Target  Return_lag1   Volume_lag1  ...  Return_lag5  \
0  0.008341       0    -0.000896  3.613550e+09  ...     0.007266   
1 -0.012867       1     0.008341  3.870520e+09  ...     0.013604   
2  0.007881       1    -0.012867  3.696660e+09  ...    -0.013421   
3  0.013813       1     0.007881  3.572660e+09  ...     0.000838   
4  0.006918       0     0.013813  3.866350e+09  ...    -0.000896   

    Volume_lag5       SMA_10       EMA_10     RSI_14       MACD  MACD_signal  \
0  4.044640e+09  1345.751001  

In [31]:
print(df.columns)

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Return', 'Target',
       'Return_lag1', 'Volume_lag1', 'Return_lag2', 'Volume_lag2',
       'Return_lag3', 'Volume_lag3', 'Return_lag4', 'Volume_lag4',
       'Return_lag5', 'Volume_lag5', 'SMA_10', 'EMA_10', 'RSI_14', 'MACD',
       'MACD_signal', 'BB_High', 'BB_Low', 'ATR_14'],
      dtype='object')


In [34]:
# Save the updated dataframe to a new CSV file
df.to_csv('Updated_file.csv', index=False)
